In [1]:
import math
import logging
import numpy as np
import pandas as pd
import scipy.signal as sig
from scipy.stats import linregress
from scipy.interpolate import InterpolatedUnivariateSpline
import scipy.optimize as opt
import altair as alt
import flammkuchen as fl
from ray import tune
from ray.tune import ProgressReporter, JupyterNotebookReporter
from ray.tune.schedulers import AsyncHyperBandScheduler, PopulationBasedTraining

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.filter_iir import Biquad
from spinorama.filter_peq import peq_build, peq_print
from spinorama.load import graph_melt
from spinorama.load_rewseq import parse_eq_iir_rews
from spinorama.graph import graph_spinorama, graph_freq, graph_regression_graph, graph_regression
from spinorama.compute_scores import scores
from spinorama.filter_scores import scores_apply_filter, scores_print, scores_loss

logger = logging.getLogger('spinorama')
fh = logging.FileHandler('debug_optim.log')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)
logger.setLevel('INFO')

In [2]:
df_all_speakers = fl.load('../cache.parse_all_speakers.h5')

In [3]:
# name of speaker
# speaker_name = 'KEF LS50'
speaker_name = 'KRK Systems Classic 5'
# all graphs for this speaker
df_speaker = df_all_speakers[speaker_name]['ASR']['asr']
# print(df_speaker.keys())
# original_mean = df_speaker['CEA2034_original_mean']
# read EQ for this speaker
my_fs = 48000
my_freq_reg_min = 300
my_freq_mean_min = 100
my_freq_mean_max = 300
manual_peq = parse_eq_iir_rews('../datas/eq/{}/iir.txt'.format(speaker_name), my_fs)
# peq_print(manual_peq)
# curve to optimise for
# curve_name = 'On Axis'
curve_name = ['Listening Window', 'Sound Power']
my_number_curves = len(curve_name)

def getFreq(df_speaker_data):
    # extract LW
    columns = {'Freq'}.union(curve_name)
    df = df_speaker_data['CEA2034_unmelted'].loc[:, columns]
    # selector
    selector = df['Freq']>my_freq_reg_min
    # freq
    freq = df.loc[selector, 'Freq'].values
    local_target = []
    for i in range(0, my_number_curves):
        local_target.append(df.loc[selector, curve_name[i]].values)
    return df, freq, local_target


def getCurve(df, freq, current_curve_name):
    selector = ((df['Freq']>my_freq_mean_min) & (df['Freq']<=my_freq_mean_max))
    # freq
    current_curve = df.loc[df['Freq']>my_freq_reg_min, current_curve_name].values
    # print(len(freq), len(current_curve))
    # compute linear reg on lw
    slope, intercept, r_value, p_value, std_err = linregress(np.log10(freq), current_curve)
    # normalise to have a flat target (primarly for bright speakers)
    #if current_curve_name in ('Listening Window', 'On Axis'):
    #    freq_100_300 = df.loc[selector, 'Freq'].values
    #    lw_100_300 = df.loc[selector, current_curve_name].values
    #    lw_100_300_mean = np.mean(lw_100_300)
    #    print('Mean {} Intercept {}'.format(lw_100_300_mean, intercept))
    #    slope = (lw_100_300_mean-intercept)/(4+math.log10(2))
    # computed on whole freq spectrum
    target_interp = [(slope*math.log10(freq[i])+intercept) for i in range(0, len(freq))]
    print('Slope {} Intercept {} R {} P {} err {}'.format(slope, intercept, r_value, p_value, std_err))
    return target_interp


# compute linear reg on lw filtered
df, freq, target = getFreq(df_all_speakers[speaker_name]['ASR']['asr'])
target_interp = []
for i in range(0, my_number_curves):
    target_interp.append(getCurve(df, freq, curve_name[i]))

#lw_interp_last = lw_interp[-1]-np.mean(lw_interp)
#if lw_interp[-1]-np.mean(lw_interp)>0.3:
#    print('Warning: bright target {}dB at {} Hz'.format(lw_interp_last, freq[-1]))

df_eq, freq_eq, lw_eq = getFreq(df_all_speakers[speaker_name]['ASR']['asr_eq'])
lw_eq_interp = []
for i in range(0, my_number_curves):
    lw_eq_interp.append(getCurve(df_eq, freq_eq, curve_name[i]))

print('manual eq loss: {}'.format(np.linalg.norm(lw_eq[0]-lw_eq_interp[0])+np.linalg.norm(lw_eq[1]-lw_eq_interp[1])))

zero = target[0][0]
alt.Chart(
    graph_melt(
        pd.DataFrame({
            'Freq': freq,
            'curve0': target[0]-zero,
            'curve0 interp': target_interp[0]-zero,
            'manual0': lw_eq[0]-lw_eq[0][0]-5,
            'manual0 interp': lw_eq_interp[0]-lw_eq[0][0]-5,
            'curve1': target[1]-zero-10,
            'curve1 interp': target_interp[1]-zero-10,
            'manual1': lw_eq[1]-lw_eq[1][0]-15,
            'manual1 interp': lw_eq_interp[1]-lw_eq[1][0]-15,
        }))
).mark_line(
    size=3
).encode(
    alt.X('Freq:Q', title='Freq (Hz)', scale=alt.Scale(type='log', nice=False, domain=[my_freq_reg_min, 20000])),
    alt.Y('dB:Q', title='Sound Pressure (dB)', scale=alt.Scale(zero=False, domain=[-40, 5])),
    alt.Color('Measurements', type='nominal', sort=None),
).properties(
    width=800,
    height=400
)

Slope 1.6882431553317083 Intercept 95.0086854641208 R 0.567837490788621 P 9.055675631647628e-12 err 0.22340576635310935
Slope -2.555619375450077 Intercept 103.31455799822102 R -0.6099972913591687 P 8.787538604671281e-14 err 0.3030567955713298
Slope -1.347987822005349 Intercept 2.6480257407463377 R -0.5124244504807981 P 1.6042223999862655e-09 err 0.2062164498630725
Slope -5.591898563421815 Intercept 10.954067224025513 R -0.8668007983015348 P 4.5533495143595883e-38 err 0.29366295125899305
manual eq loss: 32.054153616926634


alt.Chart(...)

In [4]:
logger.setLevel('INFO')
smoke_test = False
import copy

MAX_NUMBER_PEQ   = 20
MAX_STEPS_FREQ   = 5
MAX_STEPS_DBGAIN = 5
MAX_STEPS_Q      = 5
MIN_DBGAIN       = 0.5
MAX_DBGAIN       = 6
MIN_Q            = 1
MAX_Q            = 8

if smoke_test:
    MAX_NUMBER_PEQ   = 3
    MAX_STEPS_FREQ   = 3
    MAX_STEPS_DBGAIN = 3
    MAX_STEPS_Q      = 3

# main peq that we want to find    
auto_peq = []
# current target is delta between curve and a line
auto_target = []
for i in range(0, my_number_curves):
    auto_target.append(target[i]-target_interp[i])

    
# L2 norm 
def lw_loss1(local_target, peq):
    return np.linalg.norm(local_target+peq_build(freq, peq), 2)

# sum of L2 norms if we have multiple targets
def lw_loss(local_target, peq):
    return np.sum([lw_loss1(local_target[i], peq) for i in range(0, len(local_target))])

# compute pref scrore
def score_loss(local_target, peq):
    current_peq = copy.deepcopy(auto_peq)
    current_peq.append(peq)
    _, _, score = scores_apply_filter(df_speaker, current_peq)
    return score['pref_score']
    

#def lw_loss_r2(local_target, peq):
#   _, _, r_value, _, _ = linregress(np.log(freq), local_target+peq_build(freq, peq))
#   return -r_value**2


fixed_freq = set()
best_loss = lw_loss(auto_target, auto_peq)


logger.info('OPTIM {} START #PEQ {} Freq #{} Gain #{} +/-[{}, {}] Q #{} [{}, {}] Initial loss {}'.format(
    curve_name,
    MAX_NUMBER_PEQ, 
    MAX_STEPS_FREQ,
    MAX_STEPS_DBGAIN, MIN_DBGAIN, MAX_DBGAIN, 
    MAX_STEPS_Q, MIN_Q, MAX_Q, 
    best_loss))


def find_largest_area(freq, curve):
    
    def largest_area(positive_curve):
        found_peaks, _ = sig.find_peaks(positive_curve, distance=10)
        if len(found_peaks)==0:
            return None, None
        # print('found peaks at {}'.format(found_peaks))
        found_widths = sig.peak_widths(positive_curve, found_peaks, rel_height=0.1)[0]
        # print('computed width at {}'.format(found_widths))
        areas = [(i, positive_curve[found_peaks[i]]*found_widths[i]) for i in range(0, len(found_peaks))]
        # print('areas {}'.format(areas))
        sorted_areas = sorted(areas, key=lambda a: -a[1])
        # rint('sorted {}'.format(sorted_areas))
        ipeaks, area = sorted_areas[0]
        return found_peaks[ipeaks], area
        
    plus_curve = np.clip(curve, a_min=0, a_max=None)
    plus_index, plus_areas = largest_area(plus_curve)
    minus_curve = -np.clip(curve, a_min=None, a_max=0)
    minus_index, minus_areas = largest_area(minus_curve)
    
    if minus_areas is None:
        return +1, plus_index, freq[plus_index]
    
    if plus_areas is None:
        return -1, minus_index, freq[minus_index]

    if minus_areas > plus_areas:
        return -1, minus_index, freq[minus_index]
    else:
        return +1, plus_index, freq[plus_index]

    
def propose_range_freq(freq, local_target):
    sign, indice, init_freq = find_largest_area(freq, local_target)
    init_freq_min = max(init_freq*0.9, 20)
    init_freq_max = min(init_freq*1.1, 20000)
    init_freq_range = np.linspace(init_freq_min,  init_freq_max, MAX_STEPS_FREQ).tolist()
    if MAX_STEPS_FREQ == 1:
        init_freq_range = [init_freq]
    init_freq_range = np.linspace(init_freq_min,  init_freq_max, MAX_STEPS_FREQ).tolist()
    logger.debug('freq min {}Hz peak {}Hz max {}Hz'.format(init_freq_min, init_freq, init_freq_max))
    return sign, init_freq, init_freq_range


def propose_range_dbGain(freq, local_target, sign, init_freq):
    spline = InterpolatedUnivariateSpline(np.log10(freq), local_target, k=1)
    init_dbGain = abs(spline(np.log10(init_freq)))
    init_dbGain_min = max(init_dbGain/5, MIN_DBGAIN)
    init_dbGain_max = min(init_dbGain*5, MAX_DBGAIN)
    init_dbGain_range = ()
    if sign<0:
        init_dbGain_range = np.linspace(init_dbGain_min, init_dbGain_max, MAX_STEPS_DBGAIN).tolist()
    else:
        init_dbGain_range = np.linspace(-init_dbGain_max, -init_dbGain_min, MAX_STEPS_DBGAIN).tolist()
    logger.debug('gain min {}dB peak {}dB max {}dB'.format(init_dbGain_min, init_dbGain, init_dbGain_max))
    return init_dbGain_range


def propose_range_Q():
    return np.concatenate((np.linspace(MIN_Q, 1, MAX_STEPS_Q), np.linspace(1+MIN_Q, MAX_Q, MAX_STEPS_Q)), axis=0).tolist()


def propose_range_biquad():
    return [Biquad.PEAK] #, Biquad.NOTCH] #, Biquad.LOWPASS, Biquad.HIGHPASS, Biquad.BANDPASS, Biquad.LOWSHELF, Biquad.HIGHSHELF]


def find_best_biquad_raytune(auto_target, 
                             freq_range, Q_range, dbGain_range, biquad_range):

    def lw_optimizer(config, checkpoint_dir = None):
        peq = [(1.0, Biquad(config['1_type'], config['1_freq'], 48000, config['1_Q'], config['1_dbGain']))]
        intermediate_score = lw_loss(auto_target, peq)
        logger.debug('lw_optim: {} {}'.format(intermediate_score, peq_print(peq)))
        tune.report(mean_loss=intermediate_score)

    lw_analysis = tune.run(
        lw_optimizer,
        config={
            "1_freq": tune.grid_search(freq_range),
            "1_Q": tune.grid_search(Q_range),
            "1_dbGain": tune.grid_search(dbGain_range),
            "1_type": tune.grid_search(biquad_range),
        },
        progress_reporter=JupyterNotebookReporter(
            overwrite=True, 
            max_progress_rows=5, 
            max_report_frequency=30, 
            print_intermediate_tables=False
        ),
        resources_per_trial={"cpu": 1},
        metric='mean_loss',
        mode='max')
    
    best = lw_analysis.get_best_trial(metric='mean_loss', mode='min').last_result
    logger.debug('best {}'.format(best))
    best_type   = best['config']['1_type']
    best_dbGain = best['config']['1_dbGain']
    best_freq   = best['config']['1_freq']
    best_Q      = best['config']['1_Q']
    best_loss   = best['mean_loss']
    
    return True, best_type, best_freq, best_Q, best_dbGain, best_loss


def find_best_biquad_shgo(auto_target, 
                          freq_range, Q_range, dbGain_range, biquad_range):
    
    bT = 3

    def opt_peq(x):
        peq = [(1.0, Biquad(bT, x[0], 48000, x[1], x[2]))]
        return lw_loss(auto_target, peq)
                        
    bounds = [(freq_range[0], freq_range[-1]), 
              (Q_range[0], Q_range[-1]), 
              (dbGain_range[0], dbGain_range[-1])]
    
    logger.debug('Optim (shgo): range is [{}, {}], [{}, {}], [{}, {}]'.format(
        bounds[0][0], bounds[0][1], bounds[1][0], bounds[1][1], bounds[2][0], bounds[2][1]))
    res = opt.shgo(opt_peq, bounds)
    logger.info('          shgo optim loss {:2.2f} in {} iter at F {:.0f} Hz Q {:2.2f}'.format(
        res.fun, res.nfev, res.x[0], res.x[1], res.x[2]))
    return True, bT, res.x[0], res.x[1], res.x[2], res.fun
    
    
for optim_iter in range(0, MAX_NUMBER_PEQ):
    
    # target curve is currently a line between my_freq_reg_min Hz and 20kHz
    # we are optimizing above my_freq_reg_min hz on anechoic data
    auto_target = []
    for i in range(0, my_number_curves):
        auto_target.append(target[i]-target_interp[i]+peq_build(freq, auto_peq))
        
    # current loss
    # best_loss = lw_loss(auto_target, auto_peq)
    
    # greedy strategy: look for lowest & highest peak
    sign, init_freq, init_freq_range = propose_range_freq(freq, auto_target[0])
    init_dbGain_range = propose_range_dbGain(freq, auto_target[0], sign, init_freq)
    init_Q_range = propose_range_Q()
    biquad_range = propose_range_biquad()
    
    state, current_type, current_freq, current_Q, current_dbGain, current_loss = \
        find_best_biquad_raytune(auto_target, init_freq_range, init_Q_range, init_dbGain_range, biquad_range)
    if state and current_loss < best_loss:
        activate_local_optim = True
        if activate_local_optim:
            # can we optimise further with a local optimum?
            elastic = 0.8
            local_freq_range = [max(current_freq*elastic, 300), min(current_freq/elastic, 20000)]
            local_dbGain_range = []
            if current_dbGain>0:
                local_dbGain_range = [max(MIN_DBGAIN, current_dbGain*elastic), min(current_dbGain/elastic, MAX_DBGAIN)]
            else:
                local_dbGain_range = [max(-MAX_DBGAIN, current_dbGain/elastic), min(current_dbGain*elastic, -MIN_DBGAIN)]
            local_Q_range = [max(MIN_Q, current_Q*elastic), min(current_Q/elastic, MAX_Q)]
            state, local_type, local_freq, local_Q, local_dbGain, local_loss = find_best_biquad_shgo(
                auto_target, local_freq_range, local_Q_range, local_dbGain_range, biquad_range)
            if state and (local_loss < current_loss):
                logger.debug('Local optim loss {} current loss {} prev loss {}'.format(local_loss, current_loss, best_loss))
                current_type = local_type
                current_freq = local_freq
                current_dbGain = local_dbGain
                current_Q = local_Q
                current_loss = local_loss
            else:
                logger.debug('Local optim failed')
        biquad = (1.0, Biquad(current_type, current_freq, 48000, current_Q, current_dbGain))
        auto_peq.append(biquad)
        fixed_freq.add((sign, current_freq))
        best_loss = current_loss
        logger.info('Iter {:2d} Optim converged new best loss {:2.2f} biquad {:1d} F:{:5.0f}Hz Q:{:2.2f} G:{:+2.2f}dB'.format(
             optim_iter, current_loss, current_type, current_freq, current_Q, current_dbGain))
    else:
        logger.error('Skip failed optim for best {} current'.format(best_loss, current_loss))
        if activate_local_optim is False:
            activate_local_optim = True
        else:
            break

auto_target = []
for i in range(0, my_number_curves):
    auto_target.append(target[i]-target_interp[i]+peq_build(freq, auto_peq))
logger.info('OPTIM END: best loss {} with {} PEQs'.format(lw_loss(auto_target, []), len(auto_peq)))

Trial name,status,loc,1_Q,1_dbGain,1_freq,1_type,loss,iter,total time (s),neg_mean_loss
lw_optimizer_d5b3b_00000,TERMINATED,,1,-2.74019,1272.21,3,35.6902,1,0.0217073,-35.6902
lw_optimizer_d5b3b_00001,TERMINATED,,1,-2.74019,1272.21,3,35.6902,1,0.0130041,-35.6902
lw_optimizer_d5b3b_00002,TERMINATED,,1,-2.74019,1272.21,3,35.6902,1,0.0268197,-35.6902
lw_optimizer_d5b3b_00003,TERMINATED,,1,-2.74019,1272.21,3,35.6902,1,0.0226958,-35.6902
lw_optimizer_d5b3b_00004,TERMINATED,,1,-2.74019,1272.21,3,35.6902,1,0.030544,-35.6902
lw_optimizer_d5b3b_00005,TERMINATED,,2,-2.74019,1272.21,3,30.2513,1,0.0100563,-30.2513
lw_optimizer_d5b3b_00006,TERMINATED,,3.5,-2.74019,1272.21,3,26.8732,1,0.0191345,-26.8732
lw_optimizer_d5b3b_00007,TERMINATED,,5,-2.74019,1272.21,3,25.2423,1,0.0104823,-25.2423
lw_optimizer_d5b3b_00008,TERMINATED,,6.5,-2.74019,1272.21,3,24.2778,1,0.0020988,-24.2778
lw_optimizer_d5b3b_00009,TERMINATED,,8,-2.74019,1272.21,3,23.6388,1,0.0102952,-23.6388


2021-01-10 13:22:20,947	INFO tune.py:448 -- Total run time: 14.64 seconds (14.46 seconds for the tuning loop).


In [5]:
# log results        
print('Manual QE')
peq_print(manual_peq)

score = scores(df_speaker)
spin_manual, pir_manual, score_manual = scores_apply_filter(df_speaker, manual_peq)
spin_auto, pir_auto, score_auto = scores_apply_filter(df_speaker, auto_peq)

scores_print(score, score_manual)
scores_print(score, score_auto)

Manual QE
Type:3,Freq:606.0,Rate:48000.0,Q:6.0,Gain:-1.6
Type:3,Freq:652.0,Rate:48000.0,Q:15.0,Gain:2.7
Type:3,Freq:742.0,Rate:48000.0,Q:12.0,Gain:0.9
Type:3,Freq:871.0,Rate:48000.0,Q:5.5,Gain:-1.0
Type:3,Freq:987.0,Rate:48000.0,Q:13.0,Gain:-2.0
Type:3,Freq:1080.0,Rate:48000.0,Q:3.0,Gain:2.4
Type:3,Freq:1380.0,Rate:48000.0,Q:3.2,Gain:-1.9
Type:3,Freq:1570.0,Rate:48000.0,Q:14.0,Gain:1.2
Type:3,Freq:1725.0,Rate:48000.0,Q:15.0,Gain:-0.5
Type:3,Freq:1900.0,Rate:48000.0,Q:16.0,Gain:1.4
Type:3,Freq:2215.0,Rate:48000.0,Q:12.0,Gain:3.7
Type:3,Freq:2475.0,Rate:48000.0,Q:11.0,Gain:-2.0
Type:3,Freq:2990.0,Rate:48000.0,Q:8.0,Gain:-1.2
Type:3,Freq:4290.0,Rate:48000.0,Q:10.0,Gain:-0.8
Type:3,Freq:4910.0,Rate:48000.0,Q:2.3,Gain:-3.0
Type:3,Freq:7170.0,Rate:48000.0,Q:1.6,Gain:-2.9
Type:3,Freq:9650.0,Rate:48000.0,Q:3.0,Gain:-1.0
Type:3,Freq:12900.0,Rate:48000.0,Q:2.3,Gain:-1.3
Type:3,Freq:16050.0,Rate:48000.0,Q:3.5,Gain:-1.2
Type:6,Freq:19000.0,Rate:48000.0,Q:1.0,Gain:-15.0
         SPK FLT
-----------

In [ ]:
df_eq = pd.DataFrame({'Freq': freq})
for i, (pos, eq) in enumerate(manual_peq):
    df_eq['EQ {}'.format(i)] = peq_build(freq, [(pos, eq)])
    
g_eq = alt.Chart(
    graph_melt(df_eq)).mark_line().encode(
    alt.X('Freq:Q', title='Freq (Hz)', scale=alt.Scale(type='log', nice=False, domain=[my_freq_reg_min, 20000])),
    alt.Y('dB:Q', title='Sound Pressure (dB)', scale=alt.Scale(zero=False, domain=[-12,12 ])),
    alt.Color('Measurements', type='nominal', sort=None),
).properties(
    width=800,
    height=400
)

df_auto = pd.DataFrame({'Freq': freq})
for i, (pos, eq) in enumerate(auto_peq):
    df_auto['EQ {}'.format(i)] = peq_build(freq, [(pos, eq)])
    
g_auto = alt.Chart(
    graph_melt(df_auto)).mark_line().encode(
    alt.X('Freq:Q', title='Freq (Hz)', scale=alt.Scale(type='log', nice=False, domain=[my_freq_reg_min, 20000])),
    alt.Y('dB:Q', title='Sound Pressure (dB)', scale=alt.Scale(zero=False, domain=[-12,12 ])),
    alt.Color('Measurements', type='nominal', sort=None),
).properties(
    width=800,
    height=400
)

g_eq_full = alt.Chart(
    graph_melt(
        pd.DataFrame({
            'Freq': freq,
            'Manual': peq_build(freq, manual_peq),
            'Auto': peq_build(freq, auto_peq),
        }))).mark_line().encode(
    alt.X('Freq:Q', title='Freq (Hz)', scale=alt.Scale(type='log', nice=False, domain=[my_freq_reg_min, 20000])),
    alt.Y('dB:Q', title='Sound Pressure (dB)', scale=alt.Scale(zero=False, domain=[-5,5 ])),
    alt.Color('Measurements', type='nominal', sort=None),
).properties(
    width=800,
    height=400
)

g_optim = alt.Chart(
    graph_melt(
        pd.DataFrame({
            'Freq': freq,
            #'LW-Reg': lw-lw_interp,
            'Manual': lw_eq[0]-lw_eq_interp[0],
            'Auto': target[0]-target_interp[0]+peq_build(freq, auto_peq),
        }))).mark_line().encode(
    alt.X('Freq:Q', title='Freq (Hz)', scale=alt.Scale(type='log', nice=False, domain=[my_freq_reg_min, 20000])),
    alt.Y('dB:Q', title='Sound Pressure (dB)', scale=alt.Scale(zero=False, domain=[-5,5 ])),
    alt.Color('Measurements', type='nominal', sort=None),
).properties(
    width=800,
    height=400
)

((g_eq | g_auto) & (g_eq_full | g_optim)).resolve_scale('independent')

In [ ]:
zero = target[0][0]
alt.Chart(
    graph_melt(
        pd.DataFrame({
            'Freq': freq,
            'lw': lw-zero,
            'lw interp': lw_interp-zero,
            'manual': lw_eq[0]-lw_eq[0][0]-4,
            'manual interp': lw_eq_interp[0]-lw_eq[0][0]-4,
            'auto': target[0]-zero+peq_build(freq, auto_peq)-8,
            'auto interp': lw_interp[0]-zero-8,
        }))
).mark_line(
    size=3
).encode(
    alt.X('Freq:Q', title='Freq (Hz)', scale=alt.Scale(type='log', nice=False, domain=[my_freq_reg_min, 20000])),
    alt.Y('dB:Q', title='Sound Pressure (dB)', scale=alt.Scale(zero=False, domain=[-20, 0])),
    alt.Color('Measurements', type='nominal', sort=None),
).properties(
    width=800,
    height=400
)

In [ ]:
g_params = {'xmin': 20, 'xmax': 20000, 'ymin': -50, 'ymax': 10, 'width': 400, 'height': 250}

g_params['ymin'] = -40
g_params['width'] = 800
g_params['height'] = 400
graph_spinorama(df_speaker['CEA2034'], g_params).properties(title='ASR') | \
graph_spinorama(spin_manual, g_params).properties(title='ASR + manual EQ') | \
graph_spinorama(spin_auto, g_params).properties(title='ASR + auto EQ')

In [ ]:
#which_curve='Listening Window'
which_curve='Sound Power'
reg = graph_regression(df_speaker['CEA2034'].loc[(df_speaker['CEA2034'].Measurements==which_curve)], my_freq_reg_min, 20000)
origin = (graph_freq(df_speaker['CEA2034'].loc[(df_speaker['CEA2034'].Measurements==which_curve)], g_params)+reg).properties(title='ASR [{}]'.format(which_curve))
manual = (graph_freq(spin_manual.loc[(spin_manual.Measurements==which_curve)], g_params)+reg).properties(title='ASR [{}] + manual EQ'.format(which_curve))
auto = (graph_freq(spin_auto.loc[(spin_auto.Measurements==which_curve)], g_params)+reg).properties(title='ASR [{}] + auto EQ'.format(which_curve))
(origin | manual | auto)